In [1]:
import load
from plot import plot3D

# from mygp import mygp

import numpy as np
import pandas as pd
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import kernels
from sklearn.preprocessing import StandardScaler

import skopt 
from skopt.space import Real

from icecream import ic


In [2]:
class mygp(object):

    def __init__(
        self,  
        gp=None,  
        **kwargs    
    ):
    
        if gp:
            self.gp = gp    
        else:
            self.gp = GaussianProcessRegressor()    



        self.ndims=1
        self.sample_X = np.array([])
        self.sample_y = np.array([])

        
        self.kwargs = kwargs
        self.kappa = kwargs.pop("kappa",0.0)
        self.acquisition_function = kwargs.pop("acquisition_function","LCB")    
        self.xi = kwargs.pop("xi", 0.01)     


    def prn(self, **kwargs):


        print(f"ndims = {self.ndims}. sample shape = {self.sample_X.shape}")        
        print(f"kappa={self.kappa}")
        print(f"xi={self.xi}")
        print(f"acquisition_function={self.acquisition_function}")    
        print(self.sample_X)
        print(self.sample_y)   

        return

    def sample_init(self,X,y):

        X_shape = X.shape
        X_len =  len(X_shape)

        
        if X_len == 1:
            self.ndims = 1
        elif X_len == 2:
            self.ndims = X_shape[1]
        else:         
            raise ValueError(f'sample_init() sample_X has incompatiple shape"{X_shape}" for the gaussian fit method.')
        
        self.sample_X = X.copy()
        self.sample_y = y.copy()


        self.gp.fit(self.sample_X, self.sample_y)

        sample_pred , sample_std = self.gp.predict(self.sample_X, return_std=True)

        return sample_pred

    def predict(self, grid_X):
                           
        X_len = len(grid_X.shape)

        if X_len == 1 and self.ndims == 1:
            pass      
        elif X_len ==2 and grid_X.shape[1] == self.ndims:       
            pass        
        else:
            raise ValueError(f'predict() grid_X has shape "{grid_X.shape}" dimensions. ndims = {self.ndims}')   

        grid_pred, grid_std = self.gp.predict(grid_X, return_std=True)

        return grid_pred, grid_std

    def log_marginal_likelihood(self, theta=None,eval_gradient=False, clone_kernel=True):

        return self.gp.log_marginal_likelihood( theta=theta, eval_gradient=eval_gradient,clone_kernel=clone_kernel  )
    
    def get_params(self):
        ic(self.gp.get_params(deep=True))

    def sample_add(self, X1, y1):

        if len(X1) != self.ndims: 
            raise ValueError(f'sample_add(). Error sample_X1 must have exactly "{self.ndims}"')      

        # add point to array
        X = np.concatenate((self.sample_X, [X1]),axis=0 )
        y = np.concatenate((self.sample_y, [y1]),axis=0)     

        # ic(X.shape,y.shape)
        # ic(X,y)
        
        # remember if sucessfull this will update self.sample_X, self.sample_y
        self.sample_init(X,y)

        return
    
    def gp_minimize(self):

        def surrogate_function(inputs):
            # Predict using the Gaussian Process surrogate model
            y_pred, stdev = self.gp.predict(np.atleast_2d(inputs), return_std=True) 
            return y_pred[0]  



        my_domain = [Real(0.0, 1.0, name=f'num_{self.ndims}') for i in range(self.ndims)]    
        
        res = skopt.gp_minimize(
            func=surrogate_function,
            dimensions=my_domain,  # The search space
            # base_estimator= self.gp, 
            acq_func=self.acquisition_function,
            kappa= self.kappa,
            n_calls = 4,  # high to be on the safe side
            
            x0=[list(x) for x in self.sample_X], # Sample X values
            y0= self.sample_y,                   # y target 
            verbose=False                         # debug output ??
        )
        
        print(f"gp_minimize() res.x = {res.x}")
        print(f"gp_minimize() res.fun = {res.fun}")

        suggested = np.array(res.x_iters[-1])
        print("Suggested next inputs to try:", suggested)

        del res  

        return

    def op_minimize(self,X, base_estimator  ):

        if X is None:
            X=np.array([])
         
        values = np.array([])
        my_domain = [Real(0.0, 1.0, name=f'num_{self.ndims}') for i in range(self.ndims)]    

            
        # if self.acquisition_function == "LCB":
        #     acq_func_kwargs = {"kappa": self.kappa}
        # elif self.acquisition_function == "EI" or   self.acquisition_function == "PI":
        #     acq_func_kwargs = {"xi": self.xi}
 
        acq_func_kwargs = {"xi": self.xi, "kappa": self.kappa}      
          
        opt = skopt.Optimizer(
            my_domain,
            base_estimator= self.gp  if base_estimator==None else base_estimator,
            acq_func=self.acquisition_function,
            n_initial_points=1,   
            random_state=0,
            acq_func_kwargs=acq_func_kwargs,
            acq_optimizer='sampling' 
        )
        
        opt.tell(self.sample_X.tolist()   , self.sample_y.tolist(), fit=True)
        possibles = opt.ask()

        # remember opt.models[0] is the base_optimizer in opt
        
        ic(opt.models[0])
        if len(X)>0:
            match self.acquisition_function:
                case "LCB":    
                    values = skopt.acquisition.gaussian_lcb(X, opt.models[0], kappa=self.kappa )    
                case "EI": 
                    values = skopt.acquisition.gaussian_ei(X, opt.models[0], y_opt=np.min(self.sample_y), xi=self.xi )
                case "PI":
                    values = skopt.acquisition.gaussian_pi(X, opt.models[0], y_opt=np.min(self.sample_y), xi=self.xi )    
                case _:
                    raise ValueError(f"op_minimize() unknown acquisition function {self.acquisition_function}") 


        
        # print(f"op_minimize(): value = {values}")         
        # print(f"opt.ask()={possibles}")

        del opt

        return values, possibles
  




In [ ]:


def calc(idx, add):
      
    sample_X, sample_y = load.load_function(idx,add, True)
    X_plot = None

    match idx:
          
        case 1:
            # sample_y *= -1.0
            kernel = kernels.Matern(nu=2.5, length_scale=0.01 )
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=0.0, acquisition_function="LCB", xi=0.01 )
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)

        case 2:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp= mygp(g, kappa=0.0, acquisition_function="LCB",xi=0.1 )
            gp.sample_init(sample_X,sample_y)
            values, possibles=gp.op_minimize(X_plot,"GP")

        case 3: 
            # sample_y*=-1.0
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=0.5, acquisition_function="EI", xi=0.1 )
            gp.sample_init(sample_X,sample_y)
            values, possibles =gp.op_minimize(X_plot, None)

        case 4:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=1.0, acquisition_function="EI", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)          
            
        case 5:
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=1.0, acquisition_function="EI", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)
            
        case 6:
            # sample_y *= -1.0
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp= mygp(g, kappa=1.0, acquisition_function="LCB", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(X_plot, None)
        case 7:
            # sample_y *= -1.0    
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=2.0, acquisition_function="LCB", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(None, "GP")
        case 8: 
            # sample_y *= -1.0
            kernel = kernels.Matern(nu=2.5,length_scale=0.01)
            g = GaussianProcessRegressor(kernel=kernel,n_restarts_optimizer=10)
            gp = mygp(g, kappa=2.0, acquisition_function="EI", xi=0.1)
            gp.sample_init(sample_X,sample_y)
            values, possibles = gp.op_minimize(None, None)

    ic(possibles)    
    if ic.enabled:
        gp.prn()
        
    return possibles


In [7]:



ic.disable()

possibles_list=[]

for idx in range(1,9):
    print(f"idx={idx}")
    possibles=calc(idx,10)
    possibles_list.append(possibles)



load.print_for_capstone (possibles_list)


idx=1
idx=2


/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)
/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


idx=3
idx=4
idx=5
idx=6


/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


idx=7


/home/hmb/anaconda3/envs/ts/lib/python3.10/site-packages/sklearn/gaussian_process/_gpr.py:616: ConvergenceWarning: lbfgs failed to converge (status=2):
ABNORMAL_TERMINATION_IN_LNSRCH.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  _check_optimize_result("lbfgs", opt_res)


idx=8
0.628982-0.470940-
0.184758-0.018406-
0.547907-0.831788-0.460768-
0.791393-0.978545-0.935936-0.820454-
0.081432-0.401261-0.002011-0.798007-
0.001819-0.959575-0.030757-0.019916-0.950110-
0.009317-0.351462-0.541322-0.855039-0.883376-0.565164-
0.975884-0.924846-0.992106-0.273742-0.646718-0.246122-0.899852-0.403664-


Week 9 <BR>
0.281711-0.997778-<BR>
0.187256-0.153893-<BR>
0.017902-0.997712-0.776588-<BR>
0.964190-0.932673-0.908234-0.740072-<BR>
0.227669-0.640884-0.777733-0.076647-<BR>
0.147108-0.942920-0.070672-0.083941-0.950822-<BR>
0.194058-0.176859-0.970800-0.379570-0.385568-0.882665-<BR>
0.221161-0.044377-0.185963-0.124843-0.479279-0.734442-0.133208-0.259536-<BR>

0.000546-0.721183- <BR>
0.354506-0.970611- <BR>
0.989766-0.929733-0.995595-<BR>
0.994375-0.955254-0.886629-0.696208-<BR>
0.827452-0.585675-0.151086-0.400371-<BR>
0.001819-0.959575-0.030757-0.019916-0.950110-<BR>
0.289776-0.003340-0.340427-0.253258-0.446117-0.737255-<BR>
0.004768-0.075287-0.324064-0.084006-0.809979-0.791310-0.229669-0.740011-<BR>

In [5]:
ic.enable()
p = calc(2,10)

# a,b = load.load_function(1,10,True)
# print(a)
# print(b)


# ic(a.shape, b.shape)
# ic(a,b)
# print(np.min(b), np.max(b))

ic| X.shape: (10, 2), Xf.shape: (10, 2)
ic| opt.models[0]: GaussianProcessRegressor(kernel=1**2 * Matern(length_scale=[1, 1], nu=2.5) + WhiteKernel(noise_level=1),
                                            n_restarts_optimizer=2, noise='gaussian',
                                            normalize_y=True, random_state=209652396)
ic| possibles: [0.18475796274448533, 0.018406437827163117]


ndims = 2. sample shape = (28, 2)
kappa=0
xi=0.1
acquisition_function=LCB
[[6.65799580e-01 1.23969128e-01]
 [8.77790989e-01 7.78627501e-01]
 [1.42699074e-01 3.49005131e-01]
 [8.45275429e-01 7.11120267e-01]
 [4.54647141e-01 2.90455180e-01]
 [5.77712844e-01 7.71973184e-01]
 [4.38166062e-01 6.85018257e-01]
 [3.41749593e-01 2.86977198e-02]
 [3.38648157e-01 2.13867246e-01]
 [7.02636557e-01 9.26564198e-01]
 [3.31625936e-01 8.04991070e-02]
 [1.00532792e-01 4.77595237e-01]
 [6.77932161e-01 8.73703551e-01]
 [1.34955844e-01 5.21157670e-01]
 [5.37155670e-01 2.44714590e-01]
 [6.59128402e-01 6.30641565e-01]
 [4.70452072e-01 5.33768254e-01]
 [7.21202393e-01 3.64506441e-01]
 [7.22733801e-01 4.38018223e-01]
 [2.40667612e-01 1.41229074e-01]
 [7.00000000e-01 9.26000000e-01]
 [7.00000000e-01 8.26000000e-01]
 [5.00000000e-01 5.00000000e-01]
 [3.67000000e-04 5.22500000e-03]
 [3.12753000e-01 8.51616000e-01]
 [1.22140000e-01 3.18260000e-02]
 [3.54506000e-01 9.70611000e-01]
 [1.87256000e-01 1.53893000e-01]]
[